In [13]:
import sqlite3
import pandas as pd
import numpy as np
from sklearn import tree, metrics, model_selection, preprocessing
from sklearn.model_selection import KFold,cross_val_score ,cross_val_predict
from IPython.display import Image, display
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from math import sqrt
from matplotlib import pyplot as plt
%matplotlib inline  

In [15]:
cnx = sqlite3.connect('C:\MAHA\ACAD\database.sqlite') # Download data from local 
PlayerAttrib = pd.read_sql_query("Select * from Player_Attributes",cnx) 

pd.set_option('display.max_columns', None) 
pd.set_option('display.max_rows', 100)



In [16]:
print(PlayerAttrib.head())  # View the Header and Dataframe Structure
print("Dataframe PlayerAttribute Shape")
print(PlayerAttrib.shape) 
print("Dataframe PlayerAttribute Size")
print(PlayerAttrib.size)   # View the Size
print("Dataframe PlayerAttribute Columns")
print(PlayerAttrib.columns) # View Columns
print("Dataframe PlayerAttribute Summary")
print(PlayerAttrib.info())  # View Summary

   id  player_fifa_api_id  player_api_id                 date  overall_rating  \
0   1              218353         505942  2016-02-18 00:00:00            67.0   
1   2              218353         505942  2015-11-19 00:00:00            67.0   
2   3              218353         505942  2015-09-21 00:00:00            62.0   
3   4              218353         505942  2015-03-20 00:00:00            61.0   
4   5              218353         505942  2007-02-22 00:00:00            61.0   

   potential preferred_foot attacking_work_rate defensive_work_rate  crossing  \
0       71.0          right              medium              medium      49.0   
1       71.0          right              medium              medium      49.0   
2       66.0          right              medium              medium      49.0   
3       65.0          right              medium              medium      48.0   
4       65.0          right              medium              medium      48.0   

   finishing  heading_accu

In [3]:
############################### PREPROCESSING ###########################################
############ One hot Encoding 
preferred_foot_oh = pd.get_dummies(PlayerAttrib['preferred_foot'], prefix='enc')
PlayerAttrib[preferred_foot_oh.columns] = preferred_foot_oh
attacking_work_rate_oh = pd.get_dummies(PlayerAttrib['attacking_work_rate'], prefix='enc')
PlayerAttrib[attacking_work_rate_oh.columns] = attacking_work_rate_oh
defensive_work_rate_oh = pd.get_dummies(PlayerAttrib['defensive_work_rate'], prefix='enc')
PlayerAttrib[defensive_work_rate_oh.columns] = defensive_work_rate_oh

# Creating a new DF for onl Selected Attribtes for Regression 
SelPlayerAttrib = PlayerAttrib.drop(['id','player_fifa_api_id','player_api_id','date','preferred_foot',
'attacking_work_rate','defensive_work_rate' ],axis=1)

# Removing rows with Null(NaN and Infinite) Values For the Selected Player Attributes

SelPlayerAttrib = SelPlayerAttrib[(SelPlayerAttrib.overall_rating >= 0) & (SelPlayerAttrib.potential >= 0) & 
(SelPlayerAttrib.crossing >= 0) & (SelPlayerAttrib.finishing >= 0) & (SelPlayerAttrib.heading_accuracy >= 0) &
       (SelPlayerAttrib.short_passing >= 0) & (SelPlayerAttrib.volleys >= 0) & (SelPlayerAttrib.dribbling >= 0) &
       (SelPlayerAttrib.curve >= 0) & (SelPlayerAttrib.free_kick_accuracy >= 0) & (SelPlayerAttrib.long_passing >= 0) &
       (SelPlayerAttrib.ball_control >= 0) & (SelPlayerAttrib.acceleration >= 0) & (SelPlayerAttrib.sprint_speed >= 0) &
       (SelPlayerAttrib.agility >= 0) & (SelPlayerAttrib.reactions >= 0) & (SelPlayerAttrib.balance >= 0) &
       (SelPlayerAttrib.shot_power >= 0) & (SelPlayerAttrib.jumping >= 0) & (SelPlayerAttrib.stamina >= 0) &
       (SelPlayerAttrib.strength >= 0) & (SelPlayerAttrib.long_shots >= 0) & (SelPlayerAttrib.aggression >= 0) &
       (SelPlayerAttrib.interceptions >= 0) & (SelPlayerAttrib.positioning >= 0) & (SelPlayerAttrib.vision >= 0) &
       (SelPlayerAttrib.penalties >= 0) & (SelPlayerAttrib.marking >= 0) & (SelPlayerAttrib.standing_tackle >= 0) &
       (SelPlayerAttrib.sliding_tackle >= 0) & (SelPlayerAttrib.gk_diving >= 0) & (SelPlayerAttrib.gk_handling >= 0) &
       (SelPlayerAttrib.gk_kicking >= 0) & (SelPlayerAttrib.gk_positioning >= 0) & (SelPlayerAttrib.gk_reflexes >= 0)
]


# Creating a New DF to seperate Features , target
Features = SelPlayerAttrib1.drop('overall_rating',axis = 1)
Target =   SelPlayerAttrib1.overall_rating
print(Features.shape)
print(Target.shape)


# Split The Training and Test Data in 80% - 20%  corresondingly
Features_train,Features_test,Target_train,Target_test = train_test_split(Features,Target,test_size=0.20)
print(Features_train.shape)
print(Features_test.shape)
print(Target_train.shape)
print(Target_test.shape)




(25000, 61)
(25000,)
(20000, 61)
(5000, 61)
(20000,)
(5000,)


In [5]:
###################################### Building Decesion Tree Model ####################################
dtree = tree.DecisionTreeRegressor()
dtreefit = dtree.fit(Features_train, Target_train)

pred_test = dtree.predict(Features_test)

Score_train = dtreefit.score(Features_train, Target_train)
Score_test  = dtreefit.score(Features_test, Target_test)

print(" Training MSE/ Score {}".format(Score_train))
print(" Testing MSE /Score {}".format(Score_test))




 Training MSE/ Score 0.9993713932141581
 Testing MSE /Score 0.960625659801428


In [10]:
############################### Using CV Leave one out since the Training Data Score is higher than test Data (Overfitting)
dtcv_scores = cross_val_score(dtree, Features_train, Target_train, cv = 10)
scr_mean_train = dtcv_scores.mean()

print("Mean Cross Validation score on Trainin Set: ",scr_mean_train)

scr_test = dtreefit.score(Features_test, Target_test)

print(" score on Test Set: "scr_test)


# R2 on the test Set 
r2_score_test = r2_score(Target_test, pred_test)

print(" R2 Score test ",r2_score_test)



Mean Cross Validation score on Trainin Set: 0.9552596289522464
 score on Test Set: 0.960625659801428
 R2 Score test  0.960625659801
